<a href="https://colab.research.google.com/github/fowardelcac/PBI/blob/main/Notebooks/Notebooks_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! git clone https://github.com/fowardelcac/PBI.git

Cloning into 'PBI'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 37 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (37/37), 599.37 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [5]:
gdp1 = pd.read_csv('/content/PBI/PBI_dataset/Originales/gdp_data.csv')
gdp2 =  pd.read_csv('/content/PBI/PBI_dataset/Originales/country_codes.csv')
#gdp = gdp[gdp['country_name'] != 'Middle income']
df = pd.merge(gdp1, gdp2, on = 'country_code', how = 'inner')
df.head()

,country_name,country_code,year,value,region,income_group
0,Afghanistan,AFG,1960,537777811.1,South Asia,Low income
1,Afghanistan,AFG,1961,548888895.6,South Asia,Low income
2,Afghanistan,AFG,1962,546666677.8,South Asia,Low income
3,Afghanistan,AFG,1963,751111191.1,South Asia,Low income
4,Afghanistan,AFG,1964,800000044.4,South Asia,Low income


In [6]:
def graf_pbi_mundial(df):
  mundial = df.groupby('year').sum(numeric_only='True')
  fig = px.line(mundial, mundial.index, 'value')
  fig.update_layout(
      title='Crecimiento del PBI mundial desde 1960',
      xaxis_title='Año',
      yaxis_title='Valor del PBI'
  )
  fig.show()

graf_pbi_mundial(df)

In [7]:
def crecimiento_por_pais(country, df):
  df = df[df['country_name'] == country].set_index('year')

  fig = px.line(df, df.index, 'value')
  fig.update_layout(
      title = f'Crecimiento en {country} a lo largo de los años',
      xaxis_title = 'Año',
      yaxis_title = 'Valor del PBI'
  )
  fig.show()


In [8]:
crecimiento_por_pais('Argentina', df)

In [9]:
# Países miembros del G20 en inglés
g20 = [
    'Argentina',
    'Australia',
    'Brazil',
    'Canada',
    'China',
    'France',
    'Germany',
    'India',
    'Indonesia',
    'Italy',
    'Japan',
    'Mexico',
    'Russia',
    'Saudi Arabia',
    'South Africa',
    'South Korea',
    'Turkey',
    'United Kingdom',
    'United States',
]

# Países miembros del G7 en inglés
g7 = [
    'Canada',
    'France',
    'Germany',
    'Italy',
    'Japan',
    'United Kingdom',
    'United States',
]

# Países miembros del BRICS en inglés
brics = [
    'Brazil',
    'Russia',
    'India',
    'China',
    'South Africa',
]

brics_df = df[df['country_name'].isin(brics)].set_index('year')
g20_df = df[df['country_name'].isin(g20)].set_index('year')
g7_df = df[df['country_name'].isin(g7)].set_index('year')

In [10]:
def pie_por_grupo(grupo, texto):
  '''
  grupo: dataframe del grupo
  texto: nombre del grupo, g20...
  '''
  ultimo_a = grupo[grupo.index == 2022]
  fig = px.pie(ultimo_a, values='value', names='country_code')
  fig.update_layout(
      title = f'Integrantes del {texto} y su PBI',
      xaxis_title = 'Año',
      yaxis_title = 'Valor del PBI'
  )
  fig.show()

In [11]:
pie_por_grupo(g20_df, 'G20')

In [15]:
def agrupar_region_22(df):
  data_agrup_cont = df.groupby(['region','year'])['value'].sum().to_frame()
  dff = data_agrup_cont.reset_index(['region', 'year'])
  dff1 = dff[dff.year==2022]
  return dff1

In [18]:
def pbi_por_continente_barra(df):
  dff1 = agrupar_region_22(df)

  fig = px.bar(dff1, x='region', y='value')
  fig.update_layout(
      title = 'PBI por region.'
  )
  fig.show()

def pbi_porc_pie(df):
  df = agrupar_region_22(df)
  fig = px.pie(df, values='value', names='region')
  fig.update_layout(
      title = f'PBI por region',
      xaxis_title = 'Año',
      yaxis_title = 'Valor del PBI'
  )
  fig.show()



In [19]:
pbi_porc_pie(df)

In [20]:
pbi_por_continente_barra(df)

In [28]:
def pbi_hist_regi(df):
  data_agrup_cont = df.groupby(['region','year'])['value'].sum().to_frame()
  fig = px.line(data_agrup_cont.reset_index(), x='year', y='value', color='region',
                labels={'year': 'Año', 'value': 'Valor', 'region': 'Región'},
                title='Evolución del PBI por Región')

  fig.update_layout(legend=dict(x=1, y=1.2))  # Coloca la leyenda fuera del gráfico

  fig.show()

pbi_hist_regi(df)

In [29]:
def mapa(df):
  pais_index = df.loc[df.groupby('country_name')['year'].idxmax()].filter(['country_name', 'value', 'year', 'country_code'], axis = 1)

  fig = px.choropleth(
      pais_index,
      locations = 'country_code',  # Columna con códigos ISO de país
      color= 'value',          # Columna con los valores para la coloración
      hover_name = 'country_name',   # Columna con nombres de país para información al pasar el mouse
      color_continuous_scale = 'plasma',  # Escala de colores
      title = 'Mapa de Colores por PBI'
    )

  fig.update_geos(
        resolution=50,
        showcoastlines=True,
        coastlinecolor="Black",
        showland=True,
        landcolor="white",
        showocean=True,
        oceancolor="lightblue",
      )

  fig.show()

mapa(df)